# AI Prompting Techniques

This notebook demonstrates various prompting techniques for interacting with AI models, specifically using OpenAI's GPT models.

In [10]:
from openai import OpenAI
from typing import List
import json
import os
from dotenv import load_dotenv

load_dotenv()

# Define model constants
CHAT_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-small"
MAX_TOKENS = 2000
TEMPERATURE = 0.7

client: OpenAI


def initialize_openai_client() -> OpenAI:
    """Initialize and return an OpenAI client."""
    if not (api_key := os.getenv("OPENAI_API_KEY")):
        raise ValueError("OPENAI_API_KEY environment variable is not set")
    return OpenAI(api_key=api_key)


client = initialize_openai_client()

In [6]:
def get_llm_response(prompt: str, system_prompt: str = None) -> str:
    """Get a response from OpenAI's chat model."""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    else:
        messages.append(
            {
                "role": "system",
                "content": "You are a helpful assistant for the minor AI for Society.",
            }
        )
    messages.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=messages,
        max_tokens=MAX_TOKENS,
        temperature=TEMPERATURE,
    )
    return response.choices[0].message.content


def get_embedding(text: str) -> List[float]:
    """Get an embedding from the embedding model."""
    response = client.embeddings.create(input=[text], model=EMBEDDING_MODEL)
    return response.data[0].embedding


def print_prompt_and_response(technique_name: str, prompt: str, response: str):
    print(f"\n{'-'*20} {technique_name} {'-'*20}")
    print(f"Prompt:\n{prompt}")
    print(f"\nResponse:\n{response}")
    print(f"{'-'*50}\n")

## Basic Prompt Example

Basic prompting is the simplest form of interaction with an Large Language Model. It works well for straightforward questions because it allows the model to use its general knowledge without constraints. However, it may lack specificity or structure in complex scenarios.

In [7]:
def basic_prompt_example():
    question = "What are the potential impacts of AI on privacy in smart cities?"
    response = get_llm_response(question)
    print_prompt_and_response("Basic Prompt", question, response)


basic_prompt_example()


-------------------- Basic Prompt --------------------
Prompt:
What are the potential impacts of AI on privacy in smart cities?

Response:
The integration of AI in smart cities has the potential to significantly impact privacy in various ways. Here are some of the key potential impacts:

1. **Data Collection and Surveillance**: Smart cities rely on extensive data collection through sensors, cameras, and IoT devices. AI technologies analyze this data, which can lead to pervasive surveillance. This raises concerns about who has access to this data and how it is used, potentially infringing on individual privacy.

2. **Personal Data Exploitation**: AI systems often require vast amounts of personal data to function effectively. This can lead to the exploitation of personal information for commercial purposes, targeted advertising, or even discriminatory practices, undermining individual privacy.

3. **Anonymity and Identity**: AI can sometimes de-anonymize data, making it possible to iden

## Zero-Shot Learning Example

Zero-shot learning tests the AI's ability to answer questions without specific examples. This technique works because it leverages the model's general knowledge and ability to understand context. It's useful for assessing the AI's baseline capabilities on various topics.

In [16]:
def zero_shot_example():
    question = "How can AI be used to improve traffic management in smart cities?"
    prompt = f"""
    Question: {question}
    
    Please provide a detailed answer without any additional context or examples.
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Zero-Shot Learning", prompt, response)


zero_shot_example()


-------------------- Zero-Shot Learning --------------------
Prompt:

    Question: How can AI be used to improve traffic management in smart cities?
    
    Please provide a detailed answer without any additional context or examples.
    

Response:
AI can significantly enhance traffic management in smart cities through various methods, including real-time data analysis, predictive modeling, and automated decision-making systems. 

1. **Real-Time Traffic Monitoring**: AI can process data from numerous sources such as traffic cameras, sensors, GPS data from vehicles, and mobile devices. This allows for real-time monitoring of traffic conditions, identifying congestion points, and understanding traffic flow dynamics.

2. **Predictive Analytics**: By analyzing historical traffic data, AI can predict traffic patterns and potential congestion based on time of day, weather conditions, and special events. This predictive capability enables traffic management systems to proactively address 

## Structured Prompt Example
Structured prompting provides a clear format and expectations for the AI's response.
This technique works well because it guides the model to organize information in a specific way, ensuring that all required elements are addressed. It's particularly useful for complex queries that require a systematic approach.

In [11]:
def structured_prompt_example():
    question = "How can AI be used to improve sustainability in urban environments?"
    context = "Smart cities are increasingly using AI for resource management."
    format_requirements = {
        "application": "string",
        "benefits": "list",
        "challenges": "list",
    }
    evaluation_criteria = ["feasibility", "ethical considerations", "societal impact"]

    prompt = f"""
    Context: {context}
    
    Question: {question}
    
    Required Format:
    {json.dumps(format_requirements, indent=2)}
    
    Evaluation Criteria:
    {' '.join(f'- {criterion}' for criterion in evaluation_criteria)}
    
    Provide your response following exactly the format above, considering the ethical implications and societal impact.
    """

    response = get_llm_response(prompt)
    print_prompt_and_response("Structured Prompt", prompt, response)


structured_prompt_example()


-------------------- Structured Prompt --------------------
Prompt:

    Context: Smart cities are increasingly using AI for resource management.
    
    Question: How can AI be used to improve sustainability in urban environments?
    
    Required Format:
    {
  "application": "string",
  "benefits": "list",
  "challenges": "list"
}
    
    Evaluation Criteria:
    - feasibility - ethical considerations - societal impact
    
    Provide your response following exactly the format above, considering the ethical implications and societal impact.
    

Response:
{
  "application": "AI can optimize energy consumption in smart buildings by analyzing usage patterns and adjusting resources accordingly.",
  "benefits": [
    "Reduces overall energy consumption, lowering carbon emissions.",
    "Enhances the efficiency of utilities, leading to cost savings for residents and businesses.",
    "Facilitates better resource allocation, ensuring that energy supply meets demand without wastage.

## Chain of Thought Example
Chain of Thought prompting encourages the AI to break down its reasoning process. This technique is effective because it mimics human problem-solving, allowing for more transparent and logical responses. It's particularly useful for complex questions that require step-by-step analysis.

In [12]:
def chain_of_thought_example():
    question = (
        "What are the ethical considerations of using AI in healthcare diagnostics?"
    )
    prompt = f"""
    Question: {question}
    Let's approach this step-by-step:
    1) First, let's consider...
    2) Next, we should think about...
    3) Finally, we can conclude...

    Now, based on this reasoning, the answer is:
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Chain of Thought", prompt, response)


chain_of_thought_example()


-------------------- Chain of Thought --------------------
Prompt:

    Question: What are the ethical considerations of using AI in healthcare diagnostics?
    Let's approach this step-by-step:
    1) First, let's consider...
    2) Next, we should think about...
    3) Finally, we can conclude...

    Now, based on this reasoning, the answer is:
    

Response:
1) First, let's consider the accuracy and reliability of AI algorithms in healthcare diagnostics. The ethical implications here involve ensuring that AI systems are rigorously tested and validated to minimize the risk of misdiagnosis or incorrect treatment recommendations. If an AI system provides inaccurate information, it could lead to significant harm to patients, undermining the principle of "do no harm."

2) Next, we should think about issues related to data privacy and consent. AI systems often require large amounts of patient data to function effectively. It is crucial to handle this data ethically, ensuring patient co

## Few-Shot Learning Example

Few-shot learning provides the AI with examples before asking the main question.
This technique works well because it gives the model a pattern to follow, improving consistency and accuracy in responses. It's especially useful when you want the AI to mimic a specific response style or format.

In [13]:
def few_shot_example():
    question = "What is the capital of Australia?"
    prompt = f"""
    Here are a few examples:
    Q: What is the capital of Spain?
    A: The capital of Spain is Madrid.

    Q: What is the largest planet in our solar system?
    A: The largest planet in our solar system is Jupiter.

    Now, please answer the following question:
    Q: {question}
    A:
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Few-Shot Learning", prompt, response)


few_shot_example()


-------------------- Few-Shot Learning --------------------
Prompt:

    Here are a few examples:
    Q: What is the capital of Spain?
    A: The capital of Spain is Madrid.

    Q: What is the largest planet in our solar system?
    A: The largest planet in our solar system is Jupiter.

    Now, please answer the following question:
    Q: What is the capital of Australia?
    A:
    

Response:
The capital of Australia is Canberra.
--------------------------------------------------



## Role-Playing Example

Role-playing prompts the AI to adopt a specific persona or expertise.
This technique is effective because it narrows the model's focus to a particular perspective, leading to more specialized and contextually appropriate responses. It's useful for getting expert-like opinions on specific topics.

In [14]:
def role_playing_example():
    scenario = "A city is considering implementing AI-powered facial recognition in public spaces."
    role = "privacy advocate"
    system_prompt = f"You are an expert {role} working on AI applications in society. Provide responses from this perspective, considering ethical implications and societal impact."
    user_prompt = f"""
    Given the following scenario, provide your professional opinion and advice:

    Scenario: {scenario}

    Please provide your response in a professional manner, drawing from your expertise and addressing potential societal consequences.
    """
    response = get_llm_response(user_prompt, system_prompt=system_prompt)
    print_prompt_and_response(
        "Role-Playing", f"System: {system_prompt}\n\nUser: {user_prompt}", response
    )


role_playing_example()


-------------------- Role-Playing --------------------
Prompt:
System: You are an expert privacy advocate working on AI applications in society. Provide responses from this perspective, considering ethical implications and societal impact.

User: 
    Given the following scenario, provide your professional opinion and advice:

    Scenario: A city is considering implementing AI-powered facial recognition in public spaces.

    Please provide your response in a professional manner, drawing from your expertise and addressing potential societal consequences.
    

Response:
The implementation of AI-powered facial recognition technology in public spaces is a significant decision that warrants careful consideration, given its profound implications for privacy, civil liberties, and social equity.

**1. Privacy Concerns:**
Facial recognition technology operates by capturing and processing biometric data, which is inherently sensitive. The use of such technology in public spaces raises substa

## Task Decomposition Example

Task decomposition breaks down complex problems into smaller, manageable steps.
This technique works well because it helps the AI (and the user) approach complex issues systematically. It's particularly useful for planning or strategizing tasks that involve multiple components or stages.

In [15]:
def task_decomposition_example():
    complex_task = "Develop a plan to reduce carbon emissions in a major city using AI technologies."
    prompt = f"""
    Complex Task: {complex_task}

    Let's break this task down into smaller, manageable steps:

    1) [First step]
    2) [Second step]
    3) [Third step]
    ...

    Now, let's go through each step:

    1) [Explanation of first step]
    2) [Explanation of second step]
    3) [Explanation of third step]
    ...

    Conclusion: [Summary of the approach and final answer]
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Task Decomposition", prompt, response)


task_decomposition_example()


-------------------- Task Decomposition --------------------
Prompt:

    Complex Task: Develop a plan to reduce carbon emissions in a major city using AI technologies.

    Let's break this task down into smaller, manageable steps:

    1) [First step]
    2) [Second step]
    3) [Third step]
    ...

    Now, let's go through each step:

    1) [Explanation of first step]
    2) [Explanation of second step]
    3) [Explanation of third step]
    ...

    Conclusion: [Summary of the approach and final answer]
    

Response:
### Plan to Reduce Carbon Emissions in a Major City Using AI Technologies

#### Step 1: Data Collection and Analysis
- **Explanation**: The first step involves gathering and analyzing data on current carbon emissions within the city. This includes data on transportation patterns, energy consumption, industrial emissions, and waste management. AI can be utilized to process large datasets from various sources such as traffic cameras, smart meters, and environmental

## Self-Consistency Example

Self-consistency prompts the AI to generate multiple perspectives and then synthesize them. This technique is effective because it encourages a more comprehensive and balanced approach to a topic. It's particularly useful for complex issues where multiple viewpoints need to be considered.

In [17]:
def self_consistency_example():
    question = "What are the potential impacts of AI on job markets in the next decade?"
    prompt = f"""
    Question: {question}
    
    Please provide three different perspectives on this issue:
    
    Perspective 1:
    [Your response here]
    
    Perspective 2:
    [Your response here]
    
    Perspective 3:
    [Your response here]
    
    Now, synthesize these perspectives into a coherent answer:
    [Your synthesized response here]
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Self-Consistency", prompt, response)


self_consistency_example()


-------------------- Self-Consistency --------------------
Prompt:

    Question: What are the potential impacts of AI on job markets in the next decade?
    
    Please provide three different perspectives on this issue:
    
    Perspective 1:
    [Your response here]
    
    Perspective 2:
    [Your response here]
    
    Perspective 3:
    [Your response here]
    
    Now, synthesize these perspectives into a coherent answer:
    [Your synthesized response here]
    

Response:
**Perspective 1: Job Displacement and Automation**  
One of the most immediate concerns regarding AI's impact on job markets is the potential for job displacement due to automation. Many routine and repetitive tasks across various sectors, such as manufacturing, retail, and customer service, are at risk of being automated. This could lead to significant job losses for workers in these fields, especially those who may not have the skills to transition to new roles. As AI systems become more capable and co

## Constrained Generation Example
Constrained generation sets specific limits or requirements for the AI's response. This technique works well because it challenges the AI to be creative within defined boundaries, often resulting in more focused and precise outputs. It's useful when you need responses in a very specific format or style.

In [18]:
def constrained_generation_example():
    scenario = "A city is implementing an AI-powered traffic light system."
    prompt = f"""
    Scenario: {scenario}
    
    Please provide a response that meets the following constraints:
    1. Use exactly 50 words.
    2. Include at least one potential benefit and one potential risk.
    3. End with a question for further consideration.
    
    Your response:
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Constrained Generation", prompt, response)


constrained_generation_example()


-------------------- Constrained Generation --------------------
Prompt:

    Scenario: A city is implementing an AI-powered traffic light system.
    
    Please provide a response that meets the following constraints:
    1. Use exactly 50 words.
    2. Include at least one potential benefit and one potential risk.
    3. End with a question for further consideration.
    
    Your response:
    

Response:
The AI-powered traffic light system could enhance traffic flow, reducing congestion and pollution. However, it may also pose privacy risks by collecting data on vehicle movements. Balancing efficiency with privacy concerns is crucial. How can we ensure that such systems protect individual privacy while still achieving their intended benefits?
--------------------------------------------------



## Socratic Method Example - Simulation

In [26]:
def socratic_method_example():
    topic = "The impact of social media on mental health"
    initial_question = "What are the primary ways social media affects mental health?"

    questions = [initial_question]
    answers = []

    for i in range(3):  # Let's do 3 rounds of questions
        current_question = questions[-1]
        prompt = f"""
        Topic: {topic}
        Previous questions: {questions[:-1]}
        Previous answers: {answers}
        
        Current question: {current_question}
        
        Please provide a short and concise answer to the current question, and then ask a follow-up question 
        that digs deeper into the topic, challenging assumptions or exploring implications of the answer.
        
        Format your response as:
        Answer: [Your answer here]
        Thoughts: [Why you think this way]
        Possible counterarguments: [Possible counterarguments to your answer]
        Follow-up question: [Your follow-up question here]
        """

        response = get_llm_response(prompt)
        print_prompt_and_response(f"Socratic Method - Round {i+1}", prompt, response)

        # Parse the response to get the answer and next question
        answer, next_question = response.split("Follow-up question:")
        answers.append(answer.strip().replace("Answer: ", ""))
        questions.append(next_question.strip())


socratic_method_example()


-------------------- Socratic Method - Round 1 --------------------
Prompt:

        Topic: The impact of social media on mental health
        Previous questions: []
        Previous answers: []
        
        Current question: What are the primary ways social media affects mental health?
        
        Please provide a short and concise answer to the current question, and then ask a follow-up question 
        that digs deeper into the topic, challenging assumptions or exploring implications of the answer.
        
        Format your response as:
        Answer: [Your answer here]
        Thoughts: [Why you think this way]
        Possible counterarguments: [Possible counterarguments to your answer]
        Follow-up question: [Your follow-up question here]
        

Response:
Answer: Social media affects mental health primarily through increased feelings of anxiety and depression, heightened social comparison, and exposure to cyberbullying. It can also foster a sense of connec

# Reflective Prompting Example - Simulation
Reflective prompting encourages the AI to critically examine its own responses. This technique is effective because it promotes a more thoughtful and comprehensive analysis, addressing potential shortcomings in the initial response. It's particularly useful for complex topics where considering multiple perspectives and acknowledging limitations is important.

In [25]:
def reflective_prompting_example():
    initial_question = "How can AI be used to address climate change?"
    prompt = f"""
    Initial question: {initial_question}

    Please provide an initial response to this question. Then, reflect on your answer by considering:
    1. What assumptions did you make in your response?
    2. Are there any potential biases in your answer?
    3. What additional information would be helpful to provide a more comprehensive response?
    4. How might different stakeholders view this solution differently?

    After reflection, provide an updated, more nuanced response.

    Format your answer as follows:
    Initial response: [Your initial response here]

    Reflection:
    1. Assumptions: [List assumptions]
    2. Potential biases: [Discuss potential biases]
    3. Additional information needed: [List additional information]
    4. Stakeholder perspectives: [Discuss different viewpoints]

    Updated response: [Your updated, more nuanced response here]
    """
    response = get_llm_response(prompt)
    print_prompt_and_response("Reflective Prompting", prompt, response)


reflective_prompting_example()


-------------------- Reflective Prompting --------------------
Prompt:

    Initial question: How can AI be used to address climate change?

    Please provide an initial response to this question. Then, reflect on your answer by considering:
    1. What assumptions did you make in your response?
    2. Are there any potential biases in your answer?
    3. What additional information would be helpful to provide a more comprehensive response?
    4. How might different stakeholders view this solution differently?

    After reflection, provide an updated, more nuanced response.

    Format your answer as follows:
    Initial response: [Your initial response here]

    Reflection:
    1. Assumptions: [List assumptions]
    2. Potential biases: [Discuss potential biases]
    3. Additional information needed: [List additional information]
    4. Stakeholder perspectives: [Discuss different viewpoints]

    Updated response: [Your updated, more nuanced response here]
    

Response:
Initia

# Guided Feedback Prompting - Simulation

Guided feedback prompting involves providing specific feedback or guidance to the AI after its initial response. This technique allows the AI to refine and improve its answer based on targeted suggestions. It's particularly useful for obtaining more comprehensive, nuanced responses and for directing the AI's focus to specific aspects of a complex topic.

In [34]:
def guided_feedback_prompting_example():
    initial_question = (
        "What are the potential impacts of AI on job markets in the next decade?"
    )
    prompt = f"""
    Initial question: {initial_question}

    Please provide an initial response to this question.
    """
    initial_response = get_llm_response(prompt)

    feedback_prompt = f"""
    Your initial response was:
    {initial_response}

    This response is a good start, but let's improve it with the following guidance:
    1. Consider more diverse scenarios, including edge cases.
    2. Discuss both short-term and long-term risks.
    3. Include potential mitigation strategies for each risk.
    4. Address the ethical implications of these risks.

    Please respond by responding to the feedback then providing an updated response incorporating this feedback.
    """
    updated_response = get_llm_response(feedback_prompt)

    print_prompt_and_response(
        "Guided Feedback Prompting",
        f"{feedback_prompt[:300]}...\n\nUpdated Response after feedback:",
        updated_response,
    )


guided_feedback_prompting_example()


-------------------- Guided Feedback Prompting --------------------
Prompt:

    Your initial response was:
    The potential impacts of AI on job markets in the next decade are multifaceted and could lead to significant changes across various sectors. Here are some key points to consider:

1. **Job Displacement**: AI and automation may replace certain jobs, particularly th...

Updated Response after feedback:

Response:
Thank you for the feedback! It’s important to consider a broader range of scenarios and delve deeper into the risks and ethical implications associated with AI's impact on job markets. I'll address these points and enhance the original response accordingly.

### Revised Response:

The potential impacts of AI on job markets in the next decade are complex and can lead to both opportunities and challenges across various sectors. Here are some expanded considerations:

1. **Job Displacement**: 
   - **Short-term Risks**: Immediate job losses in sectors heavily reliant on 